# Hearing Data

In [2]:
using Plots
using LinearAlgebra
using Random
using DelimitedFiles

using MIDI
using MusicManipulations
using MotifSequenceGenerator

println("Imported Necessary Packages")

Imported Necessary Files


## Generating txt files of individual trajectories

In [59]:
function GenerateTrajectories_OnePath(N::Int64, JumpRates, tSteps::Int64, nTraj::Int64)
    """
    Generates trajectories data. In this code, there is only one path available. That is we start in state [0] and jump as:
        [0] -> [1] -> [2] -> ... -> [N]
    
    At each time, the atoms either jump or dont. 
    
    Note that the time step and number of steps is merely for this code. When we render the data into sound, we will take some creative liberties.
    
    @Params N: number of steps
    @Params JumpRates: Array of all the rates indexed by the state we are going to. This is a size N+1 array with entries in range (0,1), 
        but the last entry should be -1 to prevent further jumps past the last step.
    @Params tSteps: number of time steps
    @Params nTraj: number of trajectories
    """
    
    allTrajData = zeros(tSteps, nTraj) # one row for every time step. Number of columns is number of trajectories
    # we can get the jth column by taking allTrajData[:, j]
    
    for i in 1:nTraj # iterating through all of the trajectories
        states = zeros(tSteps) # this is where the current state will be stored.
        step = 1 # counds the number of steps
        currentState = 0 # number index of the states.
        JumpRate = JumpRates[currentState + 1] # Julia arrays are 1 indexed so we add 1 here.
        
        while step <= tSteps 
            #print("current step is: "); println(step)
            r = rand(Float64) # random number that dictates whether we jump or not
            if r <= JumpRate # if we jump
                #println("Jumped!")
                
                currentState = currentState + 1 # incrementing the state by 1
                JumpRate = JumpRates[currentState+1] # updating the jump rate
            end
            
            states[step] = currentState # saving the input for that array as the current state
            step = step + 1 # increment the step
        end
        #print("Current trajectory number is: "); println(i); println(states); print()
        allTrajData[:,i] = states # saves the column as the states for that run
        
    end
    #println(); println(allTrajData)
    
    ########
    # Writing into a file
    
    filePath = "RunsData/"*string(N)*"states-Trajectories"*string(nTraj)* ".txt"
    open(filePath, "w") do io
        writedlm(io, [allTrajData[j,:] for j in 1:tSteps], ',') # allTrajData[j,: ] is the jth row
    end
    # Once you read the data
    # data = readdlm("filename.txt", ',', Float64)
    # data[:,col_ind] gives you the columns
    ########
    return allTrajData
end

GenerateTrajectories_OnePath (generic function with 1 method)

In [61]:
# Generating the Trajectories

N=5
JR = zeros(N+1)
JR[1] = 0.5
JR[2] = 0.75
JR[3] = 0.99
JR[4] = 0.4
JR[5] = 0.1
JR[6] = -1.0
tS = 100
nT = 20
x=GenerateTrajectories_OnePath(N, JR, tS, nT)
x

100×20 Matrix{Float64}:
 0.0  0.0  0.0  1.0  0.0  1.0  1.0  1.0  …  0.0  0.0  1.0  0.0  1.0  1.0  1.0
 0.0  0.0  0.0  2.0  0.0  1.0  1.0  2.0     0.0  1.0  2.0  1.0  2.0  1.0  2.0
 0.0  1.0  1.0  3.0  0.0  2.0  2.0  3.0     0.0  1.0  3.0  2.0  3.0  1.0  3.0
 1.0  2.0  2.0  3.0  0.0  3.0  3.0  3.0     1.0  2.0  4.0  3.0  3.0  2.0  3.0
 2.0  3.0  3.0  3.0  0.0  3.0  4.0  4.0     2.0  3.0  4.0  3.0  3.0  3.0  3.0
 3.0  4.0  4.0  3.0  1.0  4.0  4.0  4.0  …  3.0  3.0  4.0  3.0  3.0  4.0  4.0
 3.0  4.0  4.0  4.0  2.0  4.0  4.0  4.0     4.0  4.0  4.0  3.0  3.0  4.0  4.0
 4.0  4.0  5.0  4.0  3.0  4.0  4.0  4.0     4.0  4.0  4.0  3.0  4.0  4.0  4.0
 4.0  4.0  5.0  4.0  3.0  4.0  4.0  4.0     4.0  4.0  4.0  3.0  4.0  4.0  4.0
 4.0  4.0  5.0  4.0  3.0  4.0  5.0  4.0     4.0  4.0  4.0  4.0  4.0  5.0  5.0
 4.0  4.0  5.0  5.0  3.0  4.0  5.0  4.0  …  4.0  5.0  4.0  4.0  4.0  5.0  5.0
 4.0  5.0  5.0  5.0  3.0  4.0  5.0  4.0     4.0  5.0  4.0  4.0  4.0  5.0  5.0
 4.0  5.0  5.0  5.0  3.0  4.0  5.0  4.0 

## MIDI Creation 

This link shows the MIDI notes: https://www.inspiredacoustics.com/en/MIDI_note_numbers_and_center_frequencies

The documentation has an example: https://juliamusic.github.io/JuliaMusic_documentation.jl/latest/midi/notes/

In [ ]:
function CreateMidiDiatonic_C(Data, TimeScale = 960 * 12)
    """
    Processes the trajectories data to create a midi file. I will make multiple versions of this
    This version is diatonically tuned to C. 
    
    
    @Params Data
    @Params Timescale: Length of file in ticks. 960 * 12 is 12 quarter notes
    """
    SoundFile = MIDIFile() # Midi file with tpq (ticks per quarter note) = 960
    notes = Notes() 
    
    # One track per note in C major
    trackC = MIDITrack(); trackD = MIDITrack(); trackE = MIDITrack(); trackF = MIDITrack()
    trackG = MIDITrack(); trackA = MIDITrack(); trackB = MIDITrack(); trackHighC = MIDITrack()
    
    
end

In [11]:
SoundFile = MIDIFile()  # tpq: ticks per quarter note, is default to 960. I can't seem to edit this.
track1 = MIDITrack()
notes = Notes() # vector of notes

# n = Note(pitch, velociy, position, duration) is how a note is created
# n.position gives the position. Position is given in ticks
# duration is also given in ticks 

C = Note(60, 96, 0, 192)
D = Note(62, 96, 180, 192)
push!(notes, C)
push!(notes, D)
addnotes!(track1, notes)
addtrackname!(track1, "Track 1")
push!(SoundFile.tracks, track1)
writeMIDIFile("test.mid", SoundFile)

MIDIFile (format=1, tpq=960) with tracks:
 Track 1


In [1]:
0.5 + 0.5^2 + 0.5^3 + 0.5^4 + 0.5^5 + 0.5^6 + 0.5^7

0.9921875